In [10]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

## Tuning the hyper-parameters of your estimator
- Train models with different combinations of hyper parameters
- Use cross validation to evaluate the performance of models
- Choose the best one


In [83]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [86]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(len(X_train))
print(len(X_test))

712
179


In [101]:
model = RandomForestClassifier(max_depth=100, n_estimators=10)
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.9747191011235955
testing accuracy: 0.7597765363128491


In [100]:
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.74301676, 0.80337079, 0.82022472, 0.79213483, 0.8258427 ])

You can see more classification metics here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [94]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [96]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,5,10,20]}]


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                   scoring='precision_macro')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

{'max_depth': 5, 'n_estimators': 30}
0.8420543005659284
[0.81581095 0.82214071 0.81735179 0.81318968 0.81378259 0.82602668
 0.83528764 0.8420543  0.83393485 0.81647574 0.80898731 0.81640685
 0.82181643 0.82286054 0.82359534 0.76574054 0.80066928 0.8037773
 0.81563145 0.81155712]
[0.04886118 0.03829559 0.0346351  0.03887569 0.03468483 0.03370728
 0.02906397 0.03541771 0.03670279 0.04105155 0.04620187 0.04007955
 0.02764845 0.03795026 0.02906263 0.03209155 0.01806455 0.03616352
 0.03467497 0.03229924]
[{'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 30}, {'max_depth': 3, 'n_estimators': 50}, {'max_depth': 3, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 5}, {'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 30}, {'max_depth': 5, 'n_estimators': 50}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 10, 'n_estimators': 5}, {'max_depth': 10, 'n_estimators': 10}, {'max_depth': 10, 'n_estimators': 30}

In [9]:
print("Detailed classification report:") 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:
              precision    recall  f1-score   support

           0       0.82      0.92      0.86       111
           1       0.83      0.66      0.74        68

    accuracy                           0.82       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.82      0.82      0.82       179


